#### Kmeans 군집 분석

In [1]:
import tensorflow as tf

In [2]:
sess = tf.Session()

val = [0,2,3]
val = tf.tile(val, [5])
print('홀일 때', sess.run(val))

홀일 때 [0 2 3 0 2 3 0 2 3 0 2 3 0 2 3]


In [34]:
val = [[1,1,1],[2,2,2],[3,3,3]]
val = tf.tile(val, [5,2])

print('2차원일 때\n', sess.run(val))

2차원일 때
 [[1 1 1 1 1 1]
 [2 2 2 2 2 2]
 [3 3 3 3 3 3]
 [1 1 1 1 1 1]
 [2 2 2 2 2 2]
 [3 3 3 3 3 3]
 [1 1 1 1 1 1]
 [2 2 2 2 2 2]
 [3 3 3 3 3 3]
 [1 1 1 1 1 1]
 [2 2 2 2 2 2]
 [3 3 3 3 3 3]
 [1 1 1 1 1 1]
 [2 2 2 2 2 2]
 [3 3 3 3 3 3]]


In [9]:
input_vecs = [[1.,2.],[2.,1.],[-2.,-1.]]
print('차원확대:', sess.run(tf.expand_dims(input_vecs,0))) # 열기준 차원 추가

차원확대: [[[ 1.  2.]
  [ 2.  1.]
  [-2. -1.]]]


In [10]:
print('차원확대:', sess.run(tf.expand_dims(input_vecs,1))) # 행기준 차원 추가

차원확대: [[[ 1.  2.]]

 [[ 2.  1.]]

 [[-2. -1.]]]


In [12]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn import datasets
from scipy.spatial import cKDTree
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from tensorflow.python.framework import ops

In [13]:
ops.reset_default_graph()
sess = tf.Session()
iris = datasets.load_iris()

In [25]:
num_pts = len(iris.data)
num_feats = len(iris.data[0])
k=3
generations = 25
data_points = tf.Variable(iris.data)
cluster_labels = tf.Variable(tf.zeros([num_pts], dtype=tf.int64))

In [16]:
num_pts

150

In [17]:
mum_feats

4

In [31]:
data_points

<tf.Variable 'Variable_5:0' shape=(150, 4) dtype=float64_ref>

In [21]:
# python _의 의미는, 직전에 계산 된 값을 의미
# for _ in range(k) : 회수만 사용
rand_starts = np.array([iris.data[np.random.choice(len(iris.data))] for _ in range(k)])

In [22]:
rand_starts

array([[5.2, 3.5, 1.5, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5. , 3.3, 1.4, 0.2]])

In [26]:
centroids = tf.Variable(rand_starts)

# 중심점을 행으로 반복하고 열로는 반복 x
centroid_matrix = tf.reshape(tf.tile(centroids, [num_pts, 1]), [num_pts, k, num_feats]) #행수, 군집수, 특징수(계산되는 대상)

# 들어오는 데이터 1행 3열
point_matrix = tf.reshape(tf.tile(data_points, [1,k]), [num_pts, k, num_feats])

In [30]:
centroid_matrix

<tf.Tensor 'Reshape:0' shape=(150, 3, 4) dtype=float64>

In [32]:
point_matrix

<tf.Tensor 'Reshape_1:0' shape=(150, 3, 4) dtype=float64>

In [29]:
# 3차원 빼게 되면, 행(1), 열(2), 면(0) 방향인데 axis=2 열방향으로
distances = tf.reduce_sum(tf.square(point_matrix - centroid_matrix), axis=2)

centroid_group = tf.argmin(distances, 1)

In [35]:
distances

<tf.Tensor 'Sum_1:0' shape=(150, 3) dtype=float64>

In [36]:
centroid_group

<tf.Tensor 'ArgMin:0' shape=(150,) dtype=int64>

In [37]:
# 그룹별로 중심점을 재계산하는 함수

def data_group_avg(group_ids, data):
    # 3개 그룹 요소 합계
    sum_total = tf.unsorted_segment_sum(data, group_ids, 3)
    # 그룹 계수 카운트
    num_total = tf.unsorted_segment_sum(tf.ones_like(data), group_ids, 3)
    # 새로 계산된 중심값
    avg_by_group = sum_total/num_total
    
    return avg_by_group

In [41]:
# 예제
c = tf.constant([[1,2,3,4],[5,6,7,8],[4,3,2,1]])
sess.run(tf.unsorted_segment_sum(c, tf.constant([0,1,0]), num_segments=2))

array([[5, 5, 5, 5],
       [5, 6, 7, 8]])

In [42]:
means = data_group_avg(centroid_group, data_points)
update = tf.group(centroids.assign(means), cluster_labels.assign(centroid_group))

In [43]:
means

<tf.Tensor 'truediv:0' shape=(3, 4) dtype=float64>

In [44]:
init = tf.global_variables_initializer()
sess.run(init)

for i in range(generations):
    print('Calculating gen {}, out of {}'.format(i, generations))
    _, centroid_group_count = sess.run([update, centroid_group])
    group_count = []
    
    for ix in range(k):
        group_count.append(np.sum(centroid_group_count ==ix))
    print('Group counts: {}'.format(group_count))

Calculating gen 0, out of 25
Group counts: [69, 73, 8]
Calculating gen 1, out of 25
Group counts: [73, 27, 50]
Calculating gen 2, out of 25
Group counts: [66, 34, 50]
Calculating gen 3, out of 25
Group counts: [63, 37, 50]
Calculating gen 4, out of 25
Group counts: [59, 41, 50]
Calculating gen 5, out of 25
Group counts: [54, 46, 50]
Calculating gen 6, out of 25
Group counts: [50, 50, 50]
Calculating gen 7, out of 25
Group counts: [46, 54, 50]
Calculating gen 8, out of 25
Group counts: [43, 57, 50]
Calculating gen 9, out of 25
Group counts: [40, 60, 50]
Calculating gen 10, out of 25
Group counts: [39, 61, 50]
Calculating gen 11, out of 25
Group counts: [39, 61, 50]
Calculating gen 12, out of 25
Group counts: [39, 61, 50]
Calculating gen 13, out of 25
Group counts: [39, 61, 50]
Calculating gen 14, out of 25
Group counts: [39, 61, 50]
Calculating gen 15, out of 25
Group counts: [39, 61, 50]
Calculating gen 16, out of 25
Group counts: [39, 61, 50]
Calculating gen 17, out of 25
Group counts

In [45]:
[centers, assignments] = sess.run([centroids, cluster_labels])

In [46]:
def most_common(my_list):
    return max(set(my_list), key=my_list.count)

In [47]:
label0 = most_common(list(assignments[0:50]))
label1 = most_common(list(assignments[50:100]))
label2 = most_common(list(assignments[100:150]))

group0_count = np.sum(assignments[0:50]==label0)
group1_count = np.sum(assignments[50:100]==label1)
group2_count = np.sum(assignments[100:150]==label2)

accuracy = (group0_count + group1_count + group2_count)/150.
print('Accuracy: {:.2f}'.format(accuracy))

Accuracy: 0.89
